In [258]:
from sage.crypto.util import bin_to_ascii
from sage.interfaces.ecm import *
from sage.schemes.elliptic_curves.constructor import *
def ascii_to_integer(text: str):
    """
    param: text -- исходный текст
    return: number -- числовое представление текста
    
    """
    number, r = 0, len(text)
    for i in range(r):
        number += (ord(text[i]) << (8 * i))
    return number

def er(num: int):
    """
    расширитель строки. до числа бинарных символов кратных 8.
    вход num - сумма символов в ASCII 
    выход string - строка бинарных символов длиной кратной 8.
    """
    string = str(bin(num)).replace('0b', '')
    counter = len(string) % 8
    for i in range(8 - counter):
        string = '0' + string
    return string
        

In [296]:
def ini_EC():
    """
    Создание кривой на поле GF(p), Выбор точки G и е
    выход: E - EC, G - точка на E, e - случайное число, R - поле GF(p)
    
    """
    p = random_prime(10000, lbound = 9000)
    R =  GF(p)
    e = random_prime(p - 1)
    E = EllipticCurve(R, [1, 1])
    G = E.random_point()
    return(E, G, e, R)


def commit_create(e, G, text):
    """
    Создание обязательства.
    вход: e - случайное число, G - точка на кривой, text - содержание обязательсва
    выход: C = x * G +r*H - обязательство//точка на EC
    
    """
    x = ascii_to_integer(text) # перевод текста в число
    M = x * G
    M_shtrih = e * G
    xm = int(M_shtrih[0]) #координата x точки M'
    H = xm * G
    r = random_prime(p - 1)
    C = x * G + r * H
    return C, x, r


def commit_check(C, x, r, G):
    """
    Проверка обязательства
    вход: C - обязательство, x,r - секретные данные
    выход: f'C = {C} и {x * G +xm * r * G}'
    """
    M_shtrih = x * e * G
    xm = int(M_shtrih[0])
    return f'C = {C} и {x * G + xm * r * G}'

    
def obman_boba(x, r, e, G):
    """
    плдбирает такие x' и r', что они образуют C
    C == n * G == (x + r * xm) * G
    x + r * xm = x' + r' * xm'
    
    """
    i = 1 #выход из алгоритма
    x_shtrih = 1 # новое слово
    H = e * x * G
    xm = int(H[0])
    n = (x + r * xm)
    while i != 0:
        x_shtrih += 1
        F = e * x_shtrih * G
        xm_shtrih = int(F[0])
        factors = ecm.factor(n - x_shtrih) # разложение на простые множители 
        if xm_shtrih in factors:
            i = 0
    r_shtrih = int((n - x_shtrih) / xm_shtrih)
    return x_shtrih, r_shtrih
        
        
    

In [297]:
E, G, e, R = ini_EC()
print(E)
print(f'G точка на кривой: {G}, e = {e},')
print(R)

text = '100r'

C, x, r = commit_create(e, G, text)
print()
print(f'обязаельсво С: {C}, секретные: x = {x}, r = {r}')

x_shtrih, r_shtrih =  obman_boba(x, r, e, G)
print()
print(f'новые x` = {x_shtrih} и r` = {r_shtrih} ' )

print()
print(f'проверка исходных x = {x}, r = {r}: {commit_check(C, x, r, G)}')
print(f'проверка новых    x` = {x_shtrih}, r` = {r_shtrih}: {commit_check(C, x_shtrih, r_shtrih, G)}')
print()
print(f'исходная строка: {bin_to_ascii(er(x))[::-1]}')
print(f'  новая  строка: {bin_to_ascii(er(x_shtrih)[::-1])}')

Elliptic Curve defined by y^2 = x^3 + x + 1 over Finite Field of size 9803
G точка на кривой: (3562 : 2706 : 1), e = 2239,
Finite Field of size 9803

обязаельсво С: (2006 : 2148 : 1), секретные: x = 1915760689, r = 8377

новые x` = 7644 и r` = 822679 

проверка исходных x = 1915760689, r = 8377: C = (2006 : 2148 : 1) и (1219 : 1350 : 1)
проверка новых    x` = 7644, r` = 822679: C = (2006 : 2148 : 1) и (1219 : 1350 : 1)

исходная строка: 100r
  новая  строка: ;¸
